# 演習課題その1 ベクトル空間モデル

<h3>
<p style="text-align: right;">
情報学研究科　社会情報学専攻　社会情報モデル講座　田島研究室
<br>
<br>
6930298362　修士1回　片岡 大祐
</p>
</h3>

## 必須課題（1） 与えられたコーパスに対する検索の実現

In [53]:
import json
with open("../data/kyoto_results_100.json", "r") as f:
    docs = json.load(f)
print("Num of docs = ", len(docs))

Num of docs =  83


In [54]:
import numpy as np
import gensim
from nltk.corpus import stopwords
import pandas as pd
np.set_printoptions(precision=4)

In [55]:
corpus = [doc["bow"].split() for doc in docs]

In [56]:
# stopword 除去?
# corpus

In [57]:
dictionary = gensim.corpora.Dictionary(corpus)
#dictionary.token2id

In [58]:
id_corpus = [dictionary.doc2bow(document) for document in corpus]
#id_corpus[0]

In [59]:
tfidf_model = gensim.models.TfidfModel(id_corpus, normalize=False)
tfidf_corpus = tfidf_model[id_corpus]
tfidf_vectors = gensim.matutils.corpus2dense(tfidf_corpus, len(dictionary)).T

In [60]:
# tfidf_corpus[0]

In [61]:
#[(dictionary[x[0]], x[1]) for x in tfidf_corpus[0]]

In [62]:
from scipy.spatial.distance import cosine
def cosine_sim(v1, v2):
    #scipyのcosineは類似度ではなく距離関数のため， 1-コサイン距離　とすることで，コサイン類似度に変換する
    return 1.0 - cosine(v1, v2) 

In [72]:
queries = ["京都", "京都 穴場", "関西 観光 名所"]

In [76]:
def rank_documents(query, tfidf_vectors):
    score_dic = {}
    tfidf_q = tfidf_model[dictionary.doc2bow(query.split())]  #クエリをtfidfベクトルに変換
    query_vector = gensim.matutils.corpus2dense([tfidf_q], len(dictionary)).T[0] #numpyのベクトルに変換
#     print ("q=", query_vector)
#     print(tfidf_vectors)
    for i, tfidf_vector in enumerate(tfidf_vectors):
        score_dic[i] = cosine_sim(query_vector, tfidf_vector)
    return score_dic

In [65]:
# print([(dictionary[x[0]], x[1]) for x in tfidf_q])

In [ ]:
def store_results(qid, i, doc):
    with open("../data/eval/my_q{0}.res".format(qid), "a") as resf, open("../data/eval/my_q{0}.rel".format(qid), "a") as relf:
        rel = int(input("Input the relevance of document...: "))
        relf.write("d{} L{}\n".format(i+1, rel))
        resf.write("d{}\n".format(i))

In [ ]:
qid = 1
print("-----クエリ{0}:{1}の実行結果-----\n".format(qid, str(queries[0]).strip("{''}")))
score_dic = rank_documents(queries[0], tfidf_vectors)
for i, score in sorted(score_dic.items(), key=lambda x:x[1], reverse=True)[:10]:
    print("{0}: {1} [SCORE: {2}]\n".format(i+1, "".join(corpus[i]), score))
    store_results(qid, i, "".join(corpus[i]))

In [81]:
qid = 2
print("-----クエリ{0}:{1}の実行結果-----\n".format(qid, str(queries[1]).strip("{''}")))
score_dic = rank_documents(queries[1], tfidf_vectors)
for i, score in sorted(score_dic.items(), key=lambda x:x[1], reverse=True)[:10]:
    print("{0}: {1} [SCORE: {2}]\n".format(i+1, "".join(corpus[i]), score))
    store_results(qid, i, "".join(corpus[i]))

-----クエリ2:京都 穴場の実行結果-----

61: 京都観光おすすめグルメ人気穴場京都観光おすすめ穴場スポット人気伝統行事案内しいます国際観光都市である京都四季風景食文化名所楽しめます京都各地おすすめ観光コース市バス観光バス情報 [SCORE: 4.806783218077319e+17]

Input the relevance of document...: 1
83: 京都観光おすすめ人気定番穴場観光...-NAVERまとめ京都祇園京都中人気高いエリアです魅力人気定番スポット穴場紹介しお伝えしたい思います限られた時間中京都祇園... [SCORE: 4.774427290765874e+17]

Input the relevance of document...: 1
20: 京都-定番観光名所効率良くまわる裏ワザ-〝電車...。【定番穴場おススメ京都観光スポット56USJ攻略法USJ120%楽しむ方法まとめスーツケースキャリーケース選び方日本国内死ぬ一度行きたい観光名所PRまとめのあん!?ランナーたち愛用... [SCORE: 2.8441257389579418e+17]

Input the relevance of document...: 1
37: 京都-Wikipedia京言葉京花街京町家京料理京都市内通り京都元学区京都難読地名小京都そうだ京都行こう京都東山花灯路京都嵐山花灯路京阪神京阪光源氏源氏物語外部リンク京都歴史文化データリンクbyKYOTODOTCOM [SCORE: 2.7795217684870214e+17]

Input the relevance of document...: 0
6: 京都観光研究所|京都旅行おすすめコース名所スポット地図...。京都観光研究所名所穴場150ヶ所以上観光名所掲載しいます観光名所基本データはじめ周辺スポットアクセス方法京都旅行リピーター役立つ京都観光情報地図写真... [SCORE: 0.023276471223764084]

Input the relevance of document...: 2
1: 定番穴場おススメ京都観光...-NAVERまとめ関西住んいる作者おススメ京都観光名所まとめました定番穴場スポット随時まとめいきます是非京都観光参考し下さい [SCORE: 0.

In [82]:
qid = 3
print("-----クエリ{0}:{1}の実行結果-----\n".format(qid, str(queries[2]).strip("{''}")))
score_dic = rank_documents(queries[2], tfidf_vectors)
for i, score in sorted(score_dic.items(), key=lambda x:x[1], reverse=True)[:10]:
    print("{0}: {1} [SCORE: {2}]\n".format(i+1, "".join(corpus[i]), score))
    store_results(qid, i, "".join(corpus[i]))

-----クエリ3:関西 観光 名所の実行結果-----

55: 人気名所観光スポット閲覧数ランキング-京都...-京都京都人気名所ランキング-京都観光スポット名所施設地図散策しください京都観光名所MAP散策するサイトホームGPS周辺観光地見る|キーワード検索例京都寺検索できます京都人気... [SCORE: 2.672528527273969e+17]

Input the relevance of document...: 1
68: 交通手段四条河原町観光名所人気スポット...。京都繁華街四条河原町電車バス京都市内豊富であるため交通手段選択するべき迷うところです今回四条河原町観光名所人気スポットアクセス方法まとめました [SCORE: 2.172151032271794e+17]

Input the relevance of document...: 1
77: 京都観光街めぐり～京都大好きな編集者おすすめ名所...。町家京都らしい風情感じられる建物人気高まっおり食事処宿利用されいます町家集まる美しい街並祇園有名です次読み物読む京都詳しくなれます京都... [SCORE: 2.1452621460605162e+17]

Input the relevance of document...: 1
50: 海京都天橋立観光ガイド-天橋立観光協会関西日本海側京都府北部ある日本三景天橋立公式ウェブサイト京都府北部丹後地域観光情報イベントアクセス温泉海水浴情報...626-0001京都府宮津市字文珠314番地2天橋立ターミナルセンター丹後観光... [SCORE: 1.736619571567782e+17]

Input the relevance of document...: 2
65: 京都嵐山観光おすすめ人気散策コース周辺グルメご...。京都嵐山観光おすすめ人気散策コース周辺グルメご紹介観光京都代表的な観光地桜紅葉名所なっいる嵐山一日過ごし飽きない観食べ買い物しする人気スポットです [SCORE: 0.015174791171611468]

Input the relevance of document...: 2
2: 京都観光スポットランキングTOP10-らんnet京都おすすめ観光スポット1740ヶ所セレクト人気清水寺伏見稲荷大社口コミランキングご